# Clustering

This notebook, by [felipe.alonso@urjc.es](mailto:felipe.alonso@urjc.es)

### Table of contents

1. [Kmeans](#kmeans)
2. [Hierarchical clustering](#hierarchical)
3. [DBSCAN](#dbscan)
    1. [HDBSCAN](#hdbscan)
4. [Mixture of Gaussians](#mixture)
5. [Spectral clustering](#spectral)

---
# 0. Preliminaries 

Load libraries, data sets and useful functions. A good practice is to create auxiliary functions and/or classes in additional files.

In [ ]:
# pandas, numpy and matplotlib
from src.utils import pd, np, plt

# Useful functions
from src.utils import load_examples, plot_scatter, plot_silhouette

We will use 4 synthetic data sets for illustration purposes

In [ ]:
# load data sets
X1, y1, X2, y2, X3, y3, X4, y4 = load_examples()

# Do the plotting
plt.figure(figsize = (10,10))

plt.subplot(221)
plot_scatter(X1,'3 equal sized and densed clusters')

plt.subplot(222)
plot_scatter(X2,'Unequal variance')

plt.subplot(223)
plot_scatter(X3,'Anisotropic distribution')

plt.subplot(224)
plot_scatter(X4,'Two moons')

plt.show()

---
<a id='kmeans'></a>
# 1. K-means

It is always good practice to take a look at the [user guide](https://scikit-learn.org/stable/modules/clustering.html#k-means) and [documentation](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html#sklearn.cluster.KMeans) provided by scikit-learn.

Let's start running k-means for the $X_1$, $y_1$ data set (top-left figure). We choose $k=3$ since it looks a good choice.

In [ ]:
from sklearn.cluster import KMeans

# build the clustering model
k = 3
kmeans = KMeans(n_clusters = k)
kmeans.fit(X1)

# Centroids 
centroids = kmeans.cluster_centers_

# Labels
cluster_labels = kmeans.labels_

# do the plotting
plot_scatter(X1,'k = ' + str(k), cluster_labels, centroids)
plt.show()

<div class = "alert alert-info">
<b>EXERCISE:</b> Change the value of $k$ and run the code again
</div>

We can also assign cluster labels to new points, using the predict method. Each new point is assigned to the closest cluster center when predicting, but the existing model is not changed

In [ ]:
kmeans.predict(X1)

## 1.2 How do you determine the optimal value of $k$?

Try running the algorithm for increasing $k$ and calculate the **inertia** (sum of squared distances of samples to their closest cluster center):

$$\sum_{i=0}^{n}\min_{\mu_j \in C}(||x_i - \mu_j||^2)$$

As $k$ increases, clusters become smaller, and inertia decreases. Plot this distance against the number of clusters

In [ ]:
K = range(1,10)

inertia = []
for k in K:
    kmeans = KMeans(n_clusters=k).fit(X1)
    inertia.append(kmeans.inertia_)
    
plt.plot(K,inertia,'.-')
plt.xlabel('# of clusters')
plt.ylabel('Inertia')
plt.show()

As we can see, it looks like 3 is a good candidate for the optimal value of $k$.

#### Silhouette coefficient

The [silhouette analysis](https://scikit-learn.org/stable/auto_examples/cluster/plot_kmeans_silhouette_analysis.html#sphx-glr-auto-examples-cluster-plot-kmeans-silhouette-analysis-py) can be also used to select the number of clusters.

For each sample in the data set, we can calculate a **silhouette coefficient** that measures of how close each point in one cluster is to points in the neighboring clusters. This coefficiente ranges from $-1$ to $1$, so that:
- Near +1 indicate that the sample is far away from the neighboring clusters. 
- A value of 0 indicates that the sample is on or very close to the decision boundary between two neighboring clusters and 
- Negative values indicate that those samples might have been assigned to the wrong cluster.

Silhouette coefficients can be plotted, thus providing a way to asses the number of clusters visually.

We can also calculate an **average silhouette coefficient** which gives a perspective into the density and separation of the formed clusters.

In [ ]:
k = 3
kmeans = KMeans(n_clusters=k).fit(X1)
plot_silhouette(X1,k,kmeans.labels_,kmeans.cluster_centers_)

<div class = "alert alert-info">
<b>EXERCISE:</b> For the $X_1$ data set, represent the silhouette score for different values of $k$.
</div>

**Hint**: `silhouette_avg = silhouette_score(X, cluster_labels)`

In [ ]:
# your code here
# ...

### Clustering performance evaluation

There are other metrics that we might want to calculate to analyze the quality of the clustering:

- **Cluster cardinality**: is the number of examples per cluster.
- **Cluster magnitude**: is the sum of distances from all examples to the centroid of the cluster

And many [others](https://scikit-learn.org/stable/modules/clustering.html#clustering-performance-evaluation). 

## 1.1 More examples of k-means

<div class = "alert alert-info">
<b>EXERCISE:</b> Run the K-means for $X_2$, $X_3$, $X_4$ data sets. Use inertia to determine the number of clusters.
</div>

In [ ]:
# your code here
# ...

## 1.2 K-means some conclusions

Notice that 

1. In k-means, each cluster is defined solely by its center, which means that each cluster is a convex shape. As a result of this, it can only capture relatively simple shapes. 

2. k-means also assumes that all clusters have the same “diameter” in some sense; it always draws the boundary between clusters to be exactly in the middle between the cluster centers. 

---
<a id='hierarchical'></a>
# 2. Hierarchical clustering

Let's start with an interpretable data set

In [ ]:
from sklearn.datasets import make_blobs

X, y = make_blobs(random_state=0, n_samples=12)

plt.figure(figsize=(6,6))
ax = plt.gca()
for i, x in enumerate(X):
    ax.text(x[0] + .15, x[1] + .15, "%d" % i, horizontalalignment='left', verticalalignment='center')

ax.scatter(X[:,0],X[:,1], c = 'grey', s = 150, edgecolors = 'k')
ax.set_xticks(())
ax.set_yticks(())
ax.set_aspect('equal')
plt.show()

`Scikit-learn` currently **does not have the functionality to draw dendrograms**. However, you can generate them easily using SciPy. The [SciPy clustering](https://docs.scipy.org/doc/scipy/reference/cluster.hierarchy.html#module-scipy.cluster.hierarchy) algorithms have a slightly different interface to the scikit-learn clustering algorithms. SciPy provides a function that takes a data array X and computes a linkage array, which encodes hierarchical cluster similarities. We can then feed this linkage array into the scipy dendrogram function to plot the dendrogram

In [ ]:
from scipy.cluster.hierarchy import dendrogram, linkage

Z = linkage(X, 'ward')
dendrogram(Z)
plt.show()

As we mentioned during the lecture:

- The Y-axis contains the measure of similarity between observations.
- The higher up the Y-axis, the more different two observations are.
- The height at which we cut the dendogram gives us the number of clusters.
- The same dendogram can be used to try out different numbers of clusters.


<div class = "alert alert-info">
<b>EXERCISE:</b> Take a look to the different linkage methods provided by SciPy.
</div>

Hint: [`scipy.cluster.hierarchy.linkag`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.cluster.hierarchy.linkage.html#scipy.cluster.hierarchy.linkage)


Once you decide the number of clusters you want to use using the dendrogram, you can use the scikit-learn function [AgglomerativeClustering](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.AgglomerativeClustering.html) to cluster your data (you could do it also with SciPy [fcluster](https://docs.scipy.org/doc/scipy/reference/generated/scipy.cluster.hierarchy.fcluster.html#scipy.cluster.hierarchy.fcluster)).

In [ ]:
from sklearn.cluster import AgglomerativeClustering

agg = AgglomerativeClustering(n_clusters=3).fit(X)
plot_scatter(X,'Hierarchical clustering', agg.labels_) # we do not have centroids

<div class = "alert alert-info">
<b>EXERCISE:</b> Use the dendrogram to determine the number of clusters for cases from $X_1$ to $X_4$. Then, plot the resulting clusters. Try using different linkage metrics if results are not as expected.
</div>

In [ ]:
# Your code here
# ...


---
<a id='dbscan'></a>
# 3. DBSCAN

Let’s apply DBSCAN on the synthetic dataset we used to demonstrate agglomerative clustering. Like agglomerative clustering, DBSCAN does not allow predictions on new test data, so we will use the fit_predict method to perform clustering and return the cluster labels in one step

In [ ]:
from sklearn.cluster import DBSCAN

dbscan = DBSCAN()
dbscan_labels = dbscan.fit_predict(X)
plot_scatter(X,'DBSCAN', dbscan_labels) # we do not have centroids
plt.show()
print("Cluster memberships:\n{}".format(dbscan_labels))

As you can see, all data points were assigned the label -1, which stands for noise. This is a consequence of the default parameter settings for `eps` and `min_samples`, which are not tuned for small toy datasets.

<div class = "alert alert-info">
<b>EXERCISE:</b> Try different values of $min\_samples$ and $eps$ and print the cluster membership.
</div>

In [ ]:
# your code here
# ...

Notice that:

- The parameter `eps` is somewhat more important, as it determines what it means for points to be “close.” Setting `eps` to be very small will mean that no points are core samples, and may lead to all points being labeled as noise. Setting `eps` to be very large will result in all points forming a single cluster.

- The `min_samples` setting mostly determines whether points in less dense regions will be labeled as outliers or as their own clusters. If you increase `min_samples`, anything that would have been a cluster with less than `min_samples` many samples will now be labeled as noise. `min_samples` therefore determines the minimum cluster size

<div class = "alert alert-info">
<b>EXERCISE:</b> Use DBSCAN for clustering our cases data sets: from $X_1$ to $X_4$. Try different values of $eps$ and $min\_samples$ to get the desirable cluster.
</div>

In [ ]:
# your code here
# ...

Please, note that

- When using DBSCAN, you need to be careful about handling the returned cluster assignments. **The use of -1 to indicate noise might result in unexpected effects** when using the cluster labels to index another array

<a id='hdbscan'></a>
## 3.1 HDBSCAN

First, take a look to the documentation which is available on [ReadTheDocs](http://hdbscan.readthedocs.io/en/latest/), specially to the [Parameter Selection](https://hdbscan.readthedocs.io/en/latest/parameter_selection.html#selecting-min-cluster-size) section which is summarized here:

- `min_cluster_size`: which is how big a cluster needs to be in order to form (similar to `eps` in DBSCAN). 
- `min_samples`: provide a measure of how conservative you want you clustering to be. The larger the value of min_samples you provide, the more conservative the clustering – more points will be declared as noise, and clusters will be restricted to progressively more dense areas.

Let's start with the same example as DBSCAN.

In [ ]:
import hdbscan

clusterer = hdbscan.HDBSCAN(min_cluster_size=3, min_samples = 1)
hdbscan_labels = clusterer.fit_predict(X)

plot_scatter(X,'HDBSCAN', hdbscan_labels) # we do not have centroids
plt.show()
print("Cluster memberships:\n{}".format(hdbscan_labels))

As you have seen, `min_cluster_size` and `min_samples` do not provide obvious relationship, making it difficult to set up these values, specially in real world scenarios.

<div class = "alert alert-info">
<b>EXERCISE:</b> Use HDBSCAN for clustering our cases data sets: from $X_1$ to $X_4$. Try different values of $min\_cluster\_size$ to get the desirable cluster. If results are not as expected, then vary the value of $min\_samples$.
</div>

In [ ]:
# your code here
# ...

---
<a id='mixture'></a>
# 4. Mixture of gaussians

As we mentioned, [mixture of gaussians](https://scikit-learn.org/stable/modules/generated/sklearn.mixture.GaussianMixture.html#sklearn.mixture.GaussianMixture) model attempts to find a mixture of multi-dimensional Gaussian probability distributions that best model any input data set for finding clusters.

Let's start with our $X_1$ example.

In [ ]:
from sklearn.mixture import GaussianMixture

k = 3
gmm = GaussianMixture(n_components = k).fit(X1)
gmm_labels = gmm.predict(X1)

# do the plotting
plot_scatter(X1,'k = ' + str(k), gmm_labels)
plt.show()

But because `gmm` contains a probabilistic model under the hood, it is also possible to find probabilistic cluster assignments. in Scikit-Learn this is done using the `predict_proba` method. This returns a matrix of size `[n_samples, n_clusters]` which measures the probability that any point belongs to the given cluster:

In [ ]:
gmm.predict_proba(X1).round(3)
# What if we change the number of clusters?

We can visualize the gaussian distributions fitted to the data.

In [ ]:
from src.utils import plot_gmm

k = 3
data = X1
gmm = GaussianMixture(n_components = k)
plot_gmm(gmm, data)

<div class = "alert alert-info">
<b>EXERCISE:</b> Use the above function to represent the resulting clusters for $X_2$, $X_3$ and $X_4$. Try different values of $k$.
</div>

### Covariance type

There's an extra parameter that affects the clustering process: `covariance_type`. With this parameter we control how covariance matrix is estimated and this affects in the **shape of each cluster**.

Let's see how:

In [ ]:
k = 3
data = X3
gmm = GaussianMixture(n_components = k, covariance_type = 'full')
plot_gmm(gmm, data)

- `covariance_type = 'full` allows each cluster to be modeled as an ellipse with arbitrary orientation. However, it is the most complicated and computationally expensive model (especially as the number of dimensions grows).
- `covariance_type = 'spherical` constrains the shape of the cluster such that all dimensions are equal. The resulting clustering will have similar characteristics to that of k-means, though it is not entirely equivalent.


### Number of `n_components`

The fact that `gmm` is a generative model gives us a natural means of determining the optimal number of components for a given dat set. A generative model is inherently a probability distribution for the dataset, and so we can simply evaluate the likelihood of the data under the model, using cross-validation to avoid over-fitting. 

Another means of correcting for over-fitting is to adjust the model likelihoods using some analytic criterion such as the [Akaike information criterion (AIC)](https://en.wikipedia.org/wiki/Akaike_information_criterion) or the [Bayesian information criterion (BIC)](https://en.wikipedia.org/wiki/Bayesian_information_criterion). Scikit-Learn's GMM estimator actually includes built-in methods that compute both of these, and so it is very easy to operate on this approach.

In [ ]:
n_components = np.arange(1, 21)

data = X4
models = [GaussianMixture(n,random_state=0).fit(data) for n in n_components]

plt.plot(n_components, [m.bic(data) for m in models], label='BIC')
plt.plot(n_components, [m.aic(data) for m in models], label='AIC')
plt.legend(loc='best')
plt.xlabel('n_components');

In [ ]:
k = 10
data = X4
gmm = GaussianMixture(n_components = k, covariance_type = 'full')
plot_gmm(gmm, data)

---
<a id='spectral'></a>
# 5. Spectral Clustering

Work in progress